In [1]:
# Install these packages if running from colab
!pip install tensorflow-datasets --quiet
!pip install pydot --quiet
!pip install transformers --quiet

# install huggingface datasets
!pip install datasets --quiet

! pip install rouge-score nltk --quiet
! pip install huggingface_hub --quiet

     |████████████████████████████████| 5.5 MB 24.4 MB/s 
     |████████████████████████████████| 7.6 MB 84.3 MB/s 
     |████████████████████████████████| 163 kB 85.2 MB/s 
     |████████████████████████████████| 441 kB 27.2 MB/s 
     |████████████████████████████████| 95 kB 5.9 MB/s 
     |████████████████████████████████| 212 kB 90.5 MB/s 
     |████████████████████████████████| 115 kB 88.2 MB/s 
     |████████████████████████████████| 127 kB 87.2 MB/s 
     |████████████████████████████████| 115 kB 87.1 MB/s 


In [2]:
import numpy as np
import pandas as pd

import tensorflow as tf
from tensorflow import keras

from tensorflow.keras.layers import Embedding, Input, Dense, Lambda
from tensorflow.keras.models import Model
import tensorflow.keras.backend as K
import tensorflow_datasets as tfds

import sklearn as sk
import os
import nltk
from nltk.data import find

import matplotlib.pyplot as plt

import re

#let's make longer output readable without scrolling
from pprint import pprint

# the toxic parallel dataset, with rouge metric
from datasets import load_dataset, load_from_disk, load_metric, DatasetDict

In [3]:
# Load the Drive helper and mount
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
# define paths
dataset_path = 'drive/MyDrive/Colab Notebooks/w266_project_data'
csv_path = 'drive/MyDrive/Colab Notebooks/w266_project_predictions/'

#### Change these variables as needed for different model and different file name

In [5]:
# change these variables for different models
output_file_name = 'bart_xsum_ft.csv'
model_checkpoint = "facebook/bart-large-xsum"

### Import and examine our dataset

In [6]:
# load the dataset
dataset = load_from_disk(dataset_path)

In [7]:
dataset

DatasetDict({
    train: Dataset({
        features: ['en_toxic_comment', 'en_neutral_comment'],
        num_rows: 17789
    })
    test: Dataset({
        features: ['en_toxic_comment', 'en_neutral_comment'],
        num_rows: 989
    })
    valid: Dataset({
        features: ['en_toxic_comment', 'en_neutral_comment'],
        num_rows: 988
    })
})

In [8]:
metric = load_metric("rouge")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  """Entry point for launching an IPython kernel.


In [9]:
pd.DataFrame(dataset['train'])

,en_toxic_comment,en_neutral_comment
0,u 'd be surprised all the shit u think about w...,You would be supriesd all the things you think...
1,plenty of holocaust deniers actually believe t...,Plenty of Holocaust deniers actually believe t...
2,holy fuck i 'm so going,i'm so going
3,i 've driven by lots of cattle farms but holy ...,i 've driven by lots of cattle farms but this
4,so much better this shit to soft for me,So much better this is to soft for me.
...,...,...
17784,"but , thats exactly what a couple of you idiot...","But, that’s exactly what a couple of you have ..."
17785,he probably assessed the situation and decided...,He probably assessed the situation and decided...
17786,brah im fucked up over here .,Brah I am feeling bad over here
17787,shit with my friends and not being able to tru...,with my friends and not being able to trust th...


## load BART large pretrain model

In [10]:
# model_checkpoint = "facebook/bart-large"

In [11]:
from transformers import BartTokenizer, TFBartForConditionalGeneration

model = TFBartForConditionalGeneration.from_pretrained(model_checkpoint)
tokenizer = BartTokenizer.from_pretrained(model_checkpoint)

Downloading:   0%|          | 0.00/1.51k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

All model checkpoint layers were used when initializing TFBartForConditionalGeneration.

All the layers of TFBartForConditionalGeneration were initialized from the model checkpoint at facebook/bart-large-xsum.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBartForConditionalGeneration for predictions without further training.


Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

In [12]:
model.summary()

Model: "tf_bart_for_conditional_generation"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 model (TFBartMainLayer)     multiple                  406290432 
                                                                 
 final_logits_bias (BiasLaye  multiple                 50264     
 r)                                                              
                                                                 
Total params: 406,340,696
Trainable params: 406,290,432
Non-trainable params: 50,264
_________________________________________________________________


### Preprocess: Encode detox dataset with BART tokenizer

In [13]:
## Encode detox train_texts with BART tokenizer
max_length = 25
#max_label_length = 128

def preprocess_function(examples):
    inputs = [doc for doc in examples['en_toxic_comment']]
    model_inputs = tokenizer(inputs, max_length=max_length, truncation=True, padding=True)

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            examples['en_neutral_comment'], max_length=max_length, truncation=True, padding=True
        )

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [14]:
# test the preprocess function
preprocess_function(dataset['train'][:2])

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:3547: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  "`as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your "


{'input_ids': [[0, 257, 128, 417, 28, 3911, 70, 5, 15328, 1717, 206, 59, 77, 1717, 1236, 687, 579, 2582, 179, 89, 2], [0, 2911, 24997, 9, 18701, 43328, 3069, 4733, 888, 679, 5, 5373, 15328, 51, 224, 479, 2, 1, 1, 1, 1]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0]], 'labels': [[0, 1185, 74, 28, 28836, 4458, 417, 70, 5, 383, 47, 206, 59, 77, 47, 32, 95, 2828, 89, 4, 2], [0, 16213, 24997, 9, 13903, 3069, 4733, 888, 679, 5, 383, 51, 224, 4, 2, 1, 1, 1, 1, 1, 1]]}

In [15]:
# tokenize the detox dataset
tokenized_datasets = dataset.map(preprocess_function, batched=True)

### Fine Tuning with the detox dataset

In [16]:
# hyperparameters
batch_size = 8
learning_rate = 2e-5
weight_decay = 0.01
num_train_epochs = 1

In [17]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['en_toxic_comment', 'en_neutral_comment', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 17789
    })
    test: Dataset({
        features: ['en_toxic_comment', 'en_neutral_comment', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 989
    })
    valid: Dataset({
        features: ['en_toxic_comment', 'en_neutral_comment', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 988
    })
})

In [18]:
# huggingface data collector for ROUGE
from transformers import DataCollatorForSeq2Seq
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, return_tensors="tf")
generation_data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, return_tensors="tf", pad_to_multiple_of=8)

In [19]:
# convert the train and the test tokenized dataset to TF dataset
train_dataset = model.prepare_tf_dataset(
    tokenized_datasets['train'],
    batch_size = batch_size,
    shuffle=True,
    collate_fn=data_collator,
)

validation_dataset = model.prepare_tf_dataset(
    tokenized_datasets['valid'],
    batch_size = batch_size,
    shuffle = False,
    collate_fn=data_collator,
)

test_dataset = model.prepare_tf_dataset(
    tokenized_datasets['test'],
    batch_size = batch_size,
    shuffle = False,
    collate_fn=data_collator,
)

In [20]:
# model compile
from transformers import AdamWeightDecay
import tensorflow as tf

optimizer = AdamWeightDecay(learning_rate = learning_rate, 
                            weight_decay_rate = weight_decay
                            )
model.compile(optimizer = optimizer)

No loss specified in compile() - the model's internal loss computation will be used as the loss. Don't panic - this is a common way to train TensorFlow models in Transformers! To disable this behaviour please pass a loss argument, or explicitly pass `loss=None` if you do not want your model to compute a loss.


In [21]:
# model fit
bart_model_history = model.fit(
    train_dataset, 
    validation_data = validation_dataset,
    batch_size = batch_size,
    epochs=num_train_epochs
)

2223/2223 [==============================] - 253s 96ms/step - loss: 0.9497 - val_loss: 1.0800


In [22]:
model.summary()

Model: "tf_bart_for_conditional_generation"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 model (TFBartMainLayer)     multiple                  406290432 
                                                                 
 final_logits_bias (BiasLaye  multiple                 50264     
 r)                                                              
                                                                 
Total params: 406,340,696
Trainable params: 406,290,432
Non-trainable params: 50,264
_________________________________________________________________


In [23]:
bart_model_history.history

{'loss': [0.9497366547584534], 'val_loss': [1.0799880027770996]}

### Build Rouge eval score

In [24]:
test_predictions = []
test_references = []
test_len = len(dataset['test']['en_toxic_comment'])

for i in range(test_len):
  input_tokenized = tokenizer([dataset['test']['en_toxic_comment'][i]], return_tensors="tf").input_ids
  summary_ids = model.generate(input_tokenized, num_beams=2, min_length=0, max_length=25)
  
  prediction = tokenizer.batch_decode(summary_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)
  reference = [dataset['test']['en_neutral_comment'][i]]
  
  test_predictions.insert(i, (prediction))
  test_references.insert(i, (reference))

  if i % 50 == 0:
    print('complete', i, '/', test_len)
#print(len(val_references))

complete 0 / 989
complete 50 / 989
complete 100 / 989
complete 150 / 989
complete 200 / 989
complete 250 / 989
complete 300 / 989
complete 350 / 989
complete 400 / 989
complete 450 / 989
complete 500 / 989
complete 550 / 989
complete 600 / 989
complete 650 / 989
complete 700 / 989
complete 750 / 989
complete 800 / 989
complete 850 / 989
complete 900 / 989
complete 950 / 989


In [25]:
# dictionary of lists  
dict = {'test_predictions': test_predictions, 'test_references': test_references}  
       
df = pd.DataFrame(dict) 

In [26]:
# saving the output dataframe to a csv file
# output_file_name = 'bart_large_zsl.csv'
df.to_csv(csv_path + output_file_name, index = False) 

In [27]:
# read data from csv file
df_bart_predictions = pd.read_csv(csv_path + output_file_name)

# display data frame
print(df_bart_predictions.head())

# display shape
print(df_bart_predictions.shape)

# display columns
print(df_bart_predictions.columns)

                                    test_predictions  \
0              ['The 61 61 view must be hilarious.']   
1  ['All she has to do is smile and keep her cool...   
2  ['can we apply that litmus test to muslims tha...   
3                                    ["I'm joining"]   
4       ['the residents of davis have a collective']   

                                     test_references  
0                    ['The 6161 view must be funny']  
1  ['All she has to do is smile and keep her cool...  
2  ['can we apply that litmus test to muslims tha...  
3                        ["That's it! I'm joining."]  
4    ['The residents of Davis are not so pleasant.']  
(989, 2)
Index(['test_predictions', 'test_references'], dtype='object')


In [28]:
rouge_results = metric.compute(predictions=df_bart_predictions['test_predictions'],
                               references=df_bart_predictions['test_references'])
pprint(rouge_results, compact=True)  

{'rouge1': AggregateScore(low=Score(precision=0.8018303025348583, recall=0.7990351693059717, fmeasure=0.7931453824465917), mid=Score(precision=0.816323626338844, recall=0.8122347691678447, fmeasure=0.8065879772640357), high=Score(precision=0.8298008486587539, recall=0.8252884415350382, fmeasure=0.8187691292057689)),
 'rouge2': AggregateScore(low=Score(precision=0.6827597540251041, recall=0.6759565724064924, fmeasure=0.6731520552907925), mid=Score(precision=0.7021602452164094, recall=0.6939751137031522, fmeasure=0.6911036394994527), high=Score(precision=0.7197887379335269, recall=0.7115776279017164, fmeasure=0.7090027364737693)),
 'rougeL': AggregateScore(low=Score(precision=0.7969139050532865, recall=0.7941105092853867, fmeasure=0.7883937855009956), mid=Score(precision=0.8126614046701932, recall=0.8085579103492084, fmeasure=0.8033309503171914), high=Score(precision=0.8258203191898472, recall=0.8211834487329054, fmeasure=0.8155568047834098)),
 'rougeLsum': AggregateScore(low=Score(preci

In [29]:
pd.DataFrame.from_dict(rouge_results)

,rouge1,rouge2,rougeL,rougeLsum
0,"(0.8018303025348583, 0.7990351693059717, 0.793...","(0.6827597540251041, 0.6759565724064924, 0.673...","(0.7969139050532865, 0.7941105092853867, 0.788...","(0.7982186824564658, 0.7951599921421839, 0.790..."
1,"(0.816323626338844, 0.8122347691678447, 0.8065...","(0.7021602452164094, 0.6939751137031522, 0.691...","(0.8126614046701932, 0.8085579103492084, 0.803...","(0.8121399931767148, 0.8086129566251659, 0.802..."
2,"(0.8298008486587539, 0.8252884415350382, 0.818...","(0.7197887379335269, 0.7115776279017164, 0.709...","(0.8258203191898472, 0.8211834487329054, 0.815...","(0.8264205273028746, 0.8212129212203806, 0.815..."


In [30]:
pd.concat({k: pd.DataFrame(v) for k, v in rouge_results.items()})

precision    recall  fmeasure
rouge1    0   0.801830  0.799035  0.793145
          1   0.816324  0.812235  0.806588
          2   0.829801  0.825288  0.818769
rouge2    0   0.682760  0.675957  0.673152
          1   0.702160  0.693975  0.691104
          2   0.719789  0.711578  0.709003
rougeL    0   0.796914  0.794111  0.788394
          1   0.812661  0.808558  0.803331
          2   0.825820  0.821183  0.815557
rougeLsum 0   0.798219  0.795160  0.790165
          1   0.812140  0.808613  0.802848
          2   0.826421  0.821213  0.815978